In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd

def truncate_by_words(text, num_words):
    words = text.split()
    if len(words) <= num_words:
        return text
    else:
        return " ".join(words[:num_words])

text_df = pd.read_csv("./data/0401/gemma_suppl_test.csv")
text_df['original_text'] = text_df.apply(lambda x: truncate_by_words(x['original_text'], 100), axis=1)

df1 = pd.read_csv("./data/0401/nbroad-v2.csv")
df2 = pd.read_csv("./data/0401/gemma_suppl_rewrite.csv")

df2['rewrite_prompt'] = df2['rewrite_prompts']

prompt_df = pd.concat([df1[['rewrite_prompt']], df2[['rewrite_prompt']]])
distinct_rewrite_prompts = prompt_df['rewrite_prompt'].unique()
distinct_rewrite_prompts

In [ ]:
# encode
from sentence_transformers import SentenceTransformer

t5_base = SentenceTransformer('sentence-t5-base')
#  t5_base.encode(df['original_text'].tolist()).tolist()

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def diverse_subset(texts, k):
    embeddings = t5_base.encode(texts).tolist()
    # Convert vectors to numpy array
    X = np.array(embeddings)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    
    # Get the cluster labels
    cluster_labels = kmeans.labels_
    
    # Find the centroids of each cluster
    centroids = kmeans.cluster_centers_
    
    # Initialize a list to store the diverse subset
    diverse_subset = []
    clusters = []
    
    # Select one vector from each cluster to include in the diverse subset
    for i in range(k):
        cluster_indices = np.where(cluster_labels == i)[0]
        if len(cluster_indices) > 0:
            # Choose a representative vector from the cluster (e.g., centroid)
            sorted_indices = np.argsort(np.linalg.norm(X[cluster_indices] - centroids[i], axis=1))
            # representative_index = np.argmin(np.linalg.norm(X[cluster_indices] - centroids[i], axis=1))
            # diverse_subset.append(cluster_indices[representative_index])
            for idx in sorted_indices[:1]:
                diverse_subset.append(cluster_indices[idx])
                clusters.append(i)
    
    return diverse_subset, clusters

prompts, prompt_cluster_indices = diverse_subset(distinct_rewrite_prompts, 60)
texts, _ = diverse_subset(text_df['original_text'].tolist(), 60)

prompts = [prompt_df.iloc[i]['rewrite_prompt'] for i in prompts]
texts = [text_df.iloc[i]['original_text'] for i in texts[:60]]

print(prompts.__len__())
print(texts.__len__())
print(prompt_cluster_indices.__len__())

In [ ]:
seed_df = pd.DataFrame({'prompt': prompts, 'text': texts, 'cluster': prompt_cluster_indices})
seed_df.to_csv("./data/0304/seed.csv", index=False)